In [2]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy.sparse import coo_array
from scipy import sparse
from cdlib import algorithms
from cdlib import evaluation
import sklearn
from utils import *
from distances import *
from consensus import *
import math
import itertools
import random
import time
from pathlib import Path

Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw'}


In [1]:
cons_name = "v4"

In [76]:
# DeltaSOD is calculated following the paper titled "Integrating Microarray Data by Consensus Clustering"
# by Filkov and Skiena
# Assumes the elements of the cluster are named as 0-based indices
def v4_consensus(P_list, niter=10, starting_partition=None, verbose=False):
    G = nx.Graph(P_list[0]["graph"])
    n = len(list(G.nodes()))
    k = len(P_list)
    A = nx.to_numpy_array(G)
    nz_rows, nz_cols = np.nonzero(A)
    
    t1 = time.time()
    P_list_asn = []
    c = np.zeros((n,k))
    for i in range(k):
        clust_lst = P_list[i]["partition"]
        clust_asn = clust_lst_to_asn(clust_lst)
        c[:,i] = np.array(clust_asn)
    t2 = time.time()
    print("Time to generate cluster assignment matrix:", t2-t1)
    
    t1 = time.time()
    Aw = np.array(A)
    nz_elems = []
    for i in range(len(nz_rows)):
        Aw[nz_rows[i], nz_cols[i]] = np.sum( c[nz_rows[i],:] == c[nz_cols[i],:] )
        Aw[nz_cols[i], nz_rows[i]] = Aw[nz_rows[i], nz_cols[i]]
        nz_elems.append((nz_rows[i], nz_cols[i], Aw[nz_rows[i], nz_cols[i]]))
    Gw = nx.from_numpy_array(Aw)
    nz_elems = sorted(nz_elems, key=lambda x: x[2], reverse=True)
    t2 = time.time()
    print("Time to generate weighted consensus graph:", t2-t1)
    
    t1 = time.time()
    refined_partition = None
    if starting_partition:
        refined_partition = list(starting_partition)
    else:
        refined_partition = []
        for i in range(n):
            refined_partition.append([str(i)])
    
    refined_partition_map = clust_lst_to_map(refined_partition)
    items = list(refined_partition_map.keys())
    t2 = time.time()
    print("Time to initialize:", t2-t1)
    
    tSearch = 0
    tUpdate = 0
    tMovement = 0
    count = 0
    it = 1
    while(it <= niter):
        opt_item = items[0]
        opt_deltaS = 0
        opt_a = refined_partition_map[items[0]]
        opt_b = refined_partition_map[items[0]]
        opt_x = int(opt_item)
        flag = False
        for i in range(len(nz_elems)):
            if verbose:
                print("nz_elems[", i, "]", nz_elems[i])
            t1 = time.time()
            
            x1 = nz_elems[i][0]
            x2 = nz_elems[i][1]
            
            a1 = refined_partition_map[str(x1)]
            a2 = refined_partition_map[str(x2)]
            
            Mx1a1 = 0
            Mx2a1 = 0
            Mx1a2 = 0
            Mx2a2 = 0
            for elem in refined_partition[a1]:
                if str(elem) != str(x1):
                    Mx1a1 = Mx1a1 + (k - 2 * np.sum( (c[int(x1),:] == c[int(elem),:]) ) )
                if str(elem) != str(x2):
                    Mx2a1 = Mx2a1 + (k - 2 * np.sum( (c[int(x2),:] == c[int(elem),:]) ) )
            for elem in refined_partition[a2]:
                if str(elem) != str(x1):
                    Mx1a2 = Mx1a2 + (k - 2 * np.sum( (c[int(x1),:] == c[int(elem),:]) ) )
                if str(elem) != str(x2):
                    Mx2a2 = Mx2a2 + (k - 2 * np.sum( (c[int(x2),:] == c[int(elem),:]) ) )
                    
            deltaS1 = Mx1a2 - Mx1a1
            deltaS2 = Mx2a1 - Mx2a2
            x = None
            item = None
            b = None
            deltaS = None
            if (deltaS1 < deltaS2):
                x = x1
                item = str(x1)
                a = a1
                b = a2
                deltaS = deltaS1
            else:
                x = x2
                item = str(x2)
                a = a2
                b = a1
                deltaS = deltaS2
            t2 = time.time()
            tSearch = tSearch + t2-t1
            if (deltaS is not None) and (deltaS < 0) and (a != b):
                opt_item = item
                opt_deltaS = deltaS
                opt_a = a
                opt_b = b
                opt_x = x
                
                if verbose:
                    print("---")
                    print("Move Count:", count+1, "Optimum move results in", opt_deltaS)
                    print("Move:", opt_item)
                    print("From", opt_a, ":", refined_partition[opt_a])
                    print("To", opt_b, ":", refined_partition[opt_b])
                
                t1 = time.time()
                refined_partition[opt_a].remove(opt_item)
                refined_partition[opt_b].append(opt_item)
                refined_partition_map[opt_item] = opt_b
                t2 = time.time()
                tMovement = tMovement + (t2-t1)
                if verbose:
                    print("---")
            
                count = count + 1
                flag = True
        if flag == False:
            break
        print("Iteration:", it)
        print("Move count:", count)
        it = it + 1
    print("Time to search moves:", tSearch)
    #print("Time to update M:", tUpdate)
    
    t1 = time.time()
    empty_clusters = []
    for i in range(len(refined_partition)):
        if len(refined_partition[i]) == 0:
            empty_clusters.append(i)
            
    empty_clusters.sort(reverse=True)
    for e in empty_clusters:
        del refined_partition[e]
    t2 = time.time()
    print("Time to delete empty partitions:", t2-t1)
        
    #G = nx.from_scipy_sparse_array(r)
    
    return {"graph": nx.Graph(Gw), "partition": list(refined_partition)}

# n=200

In [77]:
n = 200
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [78]:
import random

n = 200
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v4_consensus(P_list, niter=1000, starting_partition = None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)

LFR/n200/LFR_n200_mu01_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.0027289390563964844
Time to generate weighted consensus graph: 0.014474153518676758
Time to initialize: 5.984306335449219e-05
Iteration: 1
Move count: 196
Time to search moves: 1.355525255203247
Time to delete empty partitions: 6.175041198730469e-05
mu 1 , number of clusters 16
Time: 1.3950552940368652
LFR/n200/LFR_n200_mu02_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.002683401107788086
Time to generate weighted consensus graph: 0.014919042587280273
Time to initialize: 7.271766662597656e-05
Iteration: 1
Move count: 219
Time to search moves: 1.2870259284973145
Time to delete empty partitions: 3.337860107421875e-05
mu 2 , number of clusters 16
Time: 1.3167805671691895
LFR/n200/LFR_n200_mu03_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.002492666244506836
Time to generate weighted consensus graph: 0.015259742736816406
Time to initialize: 5.841255187988281e-05
Ite

# n=1000

In [79]:
n = 1000
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [80]:
import random

n = 1000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v4_consensus(P_list, niter=1000, starting_partition=None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)
                

LFR/n1000/LFR_n1000_mu01_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.012424230575561523
Time to generate weighted consensus graph: 0.1443476676940918
Time to initialize: 0.0004825592041015625
Iteration: 1
Move count: 1249
Time to search moves: 21.095488786697388
Time to delete empty partitions: 0.00014734268188476562
mu 1 , number of clusters 39
Time: 21.3703191280365
LFR/n1000/LFR_n1000_mu02_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.012740850448608398
Time to generate weighted consensus graph: 0.14628314971923828
Time to initialize: 0.0008339881896972656
Iteration: 1
Move count: 1255
Time to search moves: 22.6225745677948
Time to delete empty partitions: 0.00026679039001464844
mu 2 , number of clusters 38
Time: 22.90269923210144
LFR/n1000/LFR_n1000_mu03_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.012411355972290039
Time to generate weighted consensus graph: 0.15561676025390625
Time to initialize: 0.00041484832763671875
I

# n=5000

In [81]:
n = 5000
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [82]:
import random

n = 5000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v4_consensus(P_list, niter=1000, starting_partition = None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)
                

LFR/n5000/LFR_n5000_mu01_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.06728744506835938
Time to generate weighted consensus graph: 1.177901268005371
Time to initialize: 0.004009246826171875
Iteration: 1
Move count: 6048
Time to search moves: 107.33001971244812
Time to delete empty partitions: 0.00069427490234375
mu 1 , number of clusters 203
Time: 109.35808682441711
LFR/n5000/LFR_n5000_mu02_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.06507587432861328
Time to generate weighted consensus graph: 0.8979995250701904
Time to initialize: 0.004297494888305664
Iteration: 1
Move count: 6333
Iteration: 2
Move count: 6340
Time to search moves: 166.89000153541565
Time to delete empty partitions: 0.0007271766662597656
mu 2 , number of clusters 201
Time: 168.69196343421936
LFR/n5000/LFR_n5000_mu03_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.06515336036682129
Time to generate weighted consensus graph: 0.9263315200805664
Time to initialize: